# Install Julia - Alternative


In [1]:
# Installation cell
%%capture
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget -q 'https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

# Julia Scheme

## Utils.jl

In [ ]:

# Define the Sym function to find or create unique Symbol entries in a symbol table
function Sym(s; symbol_table=Dict{String,Symbol}())
    if !haskey(symbol_table, s)
        symbol_table[s] = Symbol(s)
    end
    return symbol_table[s]
end

# Initialize an empty dictionary to serve as the symbol table
symbol_table = Dict{String, Symbol}()

# List of predefined symbol names
symbols = ["quote", "if", "set!", "define", "lambda", "begin", "define-macro",
           "quasiquote", "unquote", "unquote-splicing"]

# Populate the symbol table with predefined symbols
for sym in symbols
    Sym(sym, symbol_table=symbol_table)
end

# Access specific symbols from the symbol table
_quote = symbol_table["quote"]
_if = symbol_table["if"]
_set = symbol_table["set!"]
_define = symbol_table["define"]
_lambda = symbol_table["lambda"]
_begin = symbol_table["begin"]
_definemacro = symbol_table["define-macro"]
_quasiquote = symbol_table["quasiquote"]
_unquote = symbol_table["unquote"]
_unquotesplicing = symbol_table["unquote-splicing"]

Symbol2("unquote-splicing")

## Exp.jl

In [ ]:
function expand(x; toplevel=false)

    # Walk tree of x, making optimizations/fixes, and signaling SyntaxError.
    require(x, x != [])  # () => Error

    if !isa(x, Array)  # constant => unchanged
        return x

    elseif x[1] === _quote
        require(x, length(x) == 2)
        return x


    elseif x[1] === _if
        if length(x) == 3
            x = [x..., nothing]  # (if t c) => (if t c nothing)
        end
        require(x, length(x) == 4, "If expression error")
        return [_if, expand(x[2], toplevel=toplevel), expand(x[3], toplevel=toplevel), expand(x[4], toplevel=toplevel)]


    elseif x[1] === _set
        require(x, length(x) == 3)
        var = x[2]  # (set! non-var exp) => Error
        require(x, isa(var, Symbol), "can set! only a symbol")
        return [_set, var, expand(x[3])]


    elseif x[1] === _define || x[1] === _definemacro
        require(x, length(x) >= 3, "Define form requires at least three parts")
        def_type, variable, body = x[1], x[2], x[3:end]

        if isa(variable, Array) && !isempty(variable)
            func_name, args = variable[1], variable[2:end]
            return expand([def_type, func_name, [_lambda, args, body...]])
        else
            require(x, isa(variable, Symbol), "Can only define symbols")
            expr = expand(body[1])
            if def_type === _definemacro
                proc = eval2(expr)  # Evaluate to get the macro procedure
                require(x, Callable(proc), "Macro must be a procedure")
                setindex!(macro_table, proc, variable)
                return nothing
            end
            return [_define, variable, expr]
        end


    elseif x[1] === _let
        require(x, length(x) >= 3, "let expression requires at least bindings and one body expression")
        bindings, body = x[2], x[3:end]

        new_bindings = map(b -> begin
            require(b, isa(b, Vector) && length(b) == 2 && isa(b[1], Symbol2), "Invalid let binding format")
            return (b[1], expand(b[2], toplevel=toplevel))
        end, bindings)

        expanded_body = map(b -> expand(b, toplevel=toplevel), [body])
        return let2(new_bindings, expanded_body...)


    elseif x[1] === _begin
        if length(x) == 1
            return nothing  # (begin) => nothing
        else
            return [expand(xi) for xi in x]
        end


    elseif x[1] === _lambda  # (lambda (x) e1 e2) => (lambda (x) (begin e1 e2))
        require(x, length(x) >= 3)
        vars, body = x[2], x[3:end]
        require(x, (isa(vars, Array) && all(v -> isa(v, Symbol2), vars)) || isa(vars, Symbol2), "illegal lambda argument list")

        exp = length(body) == 1 ? body[1] : vcat([_begin], body)
        return [_lambda, vars, expand(exp)]


    elseif x[1] === _quasiquote  # `x => expand_quasiquote(x)
        require(x, length(x) == 2)
        return expand_quasiquote(x[2])


    elseif isa(x[1], Symbol2) && (x[1] ∈ keys(macro_table))
        return expand((macro_table[x[1]])(x[2:end]...))


    else  # (f arg...) => expand each
        return map(expand, x)
    end
end



function require(x, predicate::Bool, msg="wrong length")
    if !predicate
        error(to_string(x) * ": " * msg)
    end
end

_append, _cons, _let = Symbol2("append"), Symbol2("cons"), Symbol2("let")

function is_pair(x)
    isa(x, Array) && !isempty(x)
end


function expand_quasiquote(x)
    """Expand `x => 'x; `,x => x; `(,@x y) => (append x y) """
    if !is_pair(x)
        return [_quote, x]
    end
    require(x, x[1] !== _unquotesplicing, "can't splice here")
    if x[1] === _unquote
        require(x, length(x) == 2)
        return x[2]
    elseif is_pair(x[1]) && x[1][1] === _unquotesplicing
        require(x[1], length(x[1]) == 2)
        return [_append, x[1][2], expand_quasiquote(x[2:end])]
    else
        return [_cons, expand_quasiquote(x[1]), expand_quasiquote(x[2:end])]
    end
end


function to_string(x)
    if x === true
        return "#t"
    elseif x === false
        return "#f"
    elseif isa(x, Symbol2)  # Assuming Symbol2 is used for custom symbols
        return x.name
    elseif isa(x, String)
        # Use Julia's escape_string to handle special characters
        return "\"" * escape_string(x) * "\""
    elseif isa(x, Array)
        # Recursively convert array elements to strings and join with spaces
        return "(" * join(map(to_string, x), " ") * ")"
    elseif isa(x, Complex)
        # Replace Julia's 'im' with 'i' for complex numbers
        return replace(string(x), "im" => "i")
    else
        return string(x)
    end
end


function let2(bindings, body)
    require(bindings, all(b -> isa(b, Tuple) && length(b) == 2 && isa(b[1], Symbol2), bindings), "let2 expects bindings as tuples of (Symbol2, expression)")

    # Unpack the variables and expressions from bindings
    vars = [b[1] for b in bindings]
    vals = [expand(b[2]) for b in bindings]  # Ensure values are expanded

    # Prepare the body: if there are multiple expressions, wrap them in a 'begin'
    expanded_body = length(body) > 1 ? map(expand, body) : expand(body[1])

    # Construct the lambda expression
    lambda_expr = [_lambda, vars, expanded_body]

    return [lambda_expr, vals...]
end

macro_table = Dict{Any, Any}(Sym("let") => let2)

function unzip(bindings)
    vars = [b[1] for b in bindings]
    vals = [b[2] for b in bindings]
    return vars, vals
end


function Callable(x)
    # Check if the typeof(x) has any methods defined for call syntax
    return !isempty(methods(x))
end

Callable (generic function with 1 method)

In [ ]:
using Test

@testset "Scheme Interpreter Tests" begin
    @testset "expand function tests" begin
        @test begin
            result = expand(42)
            println("expand(42) gives the result: $result \n")
            result == 42
        end

        @test begin
            result = expand([_quote, Sym("exp")])
            println("expand([_quote, 'exp']) gives the result: $result \n")
            result == [_quote, Sym("exp")]
        end

        @test begin
            result = expand([_if, Sym("test"), Sym("consequence"), Sym("alternative")])
            println("expand([_if, 'test', 'consequence', 'alternative']) gives the result: $result \n")
            result == [_if, Sym("test"), Sym("consequence"), Sym("alternative")]
        end

        @test begin
            result = expand([_if, Sym("test"), Sym("consequence")])
            println("expand([_if, 'test', 'consequence']) gives the result: $result \n")
            result == [_if, Sym("test"), Sym("consequence"), nothing]
        end

        @test begin
            result = expand([_set, Sym("var"), 42])
            println("expand([_set, Sym('var'), 42]) gives the result: $result \n")
            result == [_set, Sym("var"), 42]
        end

        @test begin
            result = expand([_define, Sym("var"), 42])
            println("expand([_define, Sym('var'), 42]) gives the result: $result \n")
            result == [_define, Sym("var"), 42]
        end

        @test begin
            result = expand([_define, [Sym("f"), Sym("arg")], "body"])
            println("expand([_define, [Sym('f'), Sym('arg')], 'body']) gives the result: $result \n")
            result == [_define, Sym("f"), [_lambda, [Sym("arg")], "body"]]
        end

        @test begin
            result = expand([_begin, 42, Sym("exp")])
            println("expand([_begin, 42, 'exp']) gives the result: $result \n")
            result == [_begin, 42, Sym("exp")]
        end

        @test begin
            result = expand([_lambda, [Sym("arg")], Sym("body")])
            println("expand([_lambda, [Sym('arg')], 'body']) gives the result: $result \n")
            result == [_lambda, [Sym("arg")], Sym("body")]
        end

        @test begin
            result = expand_quasiquote([_unquote, Sym("exp")])
            println("expand_quasiquote([_unquote, 'exp']) gives the result: $result \n")
            result == Sym("exp")
        end


        @test begin
            result = let2([(Sym("x"), 42), (Sym("y"), Sym("exp"))], "body")
            println("let2([[Sym('x'), 42], [Sym('y'), 'exp']], 'body') gives the result: $result \n")
            result == [[_lambda, [Sym("x"), Sym("y")], "body"], 42, Sym("exp")]
        end
    end
end

expand(42) gives the result: 42 

expand([_quote, 'exp']) gives the result: Symbol2[Symbol2("quote"), Symbol2("exp")] 

expand([_if, 'test', 'consequence', 'alternative']) gives the result: Symbol2[Symbol2("if"), Symbol2("test"), Symbol2("consequence"), Symbol2("alternative")] 

expand([_if, 'test', 'consequence']) gives the result: Union{Nothing, Symbol2}[Symbol2("if"), Symbol2("test"), Symbol2("consequence"), nothing] 

expand([_set, Sym('var'), 42]) gives the result: Any[Symbol2("set!"), Symbol2("var"), 42] 

expand([_define, Sym('var'), 42]) gives the result: Any[Symbol2("define"), Symbol2("var"), 42] 

expand([_define, [Sym('f'), Sym('arg')], 'body']) gives the result: Any[Symbol2("define"), Symbol2("f"), Any[Symbol2("lambda"), Symbol2[Symbol2("arg")], "body"]] 

expand([_begin, 42, 'exp']) gives the result: Any[Symbol2("begin"), 42, Symbol2("exp")] 

expand([_lambda, [Sym('arg')], 'body']) gives the result: Any[Symbol2("lambda"), Symbol2[Symbol2("arg")], Symbol2("body")] 

expand

Test.DefaultTestSet("Scheme Interpreter Tests", Any[Test.DefaultTestSet("expand function tests", Any[], 11, false, false)], 0, false, false)

## Parse.jl

In [ ]:
using Base: readline, match

# EOF object representation
const eof_object = Symbol2("#<eof-object>")

# Tokenizer pattern
tokenizer = r"""\s*(,@|[('`,)]|"(?:[\\].|[^\\"])*"|;.*|[^\s('"`,;)]*)(.*)"""

# InPort class equivalent in Julia
mutable struct InPort
    file::IO
    line::String
    InPort(file::IO) = new(file, "")
end

function next_token(inport::InPort)
    while true
        if isempty(inport.line)
            inport.line = readline(inport.file, keep=true)
        end

        if isempty(inport.line)
            return eof_object
        end

        match_result = Base.match(tokenizer, inport.line)  # Use Base.match explicitly
        if match_result !== nothing
            token, inport.line = match_result.captures
            if !isempty(token) && !startswith(token, ";")
                return token
            end
        end
    end
end

# Quotes mapping
quotes = Dict("'" => _quote, "`" => _quasiquote, "," => _unquote, ",@" => _unquotesplicing)

function read2(inport::InPort)
    function read_ahead(token)
        if token == "("
            L = []
            while true
                token = next_token(inport)
                if token == ")"
                    return L
                else
                    push!(L, read_ahead(token))
                end
            end
        elseif token == ")"
            error("unexpected )")
        elseif haskey(quotes, token)
            return [quotes[token], read2(inport)]
        elseif token === eof_object
            error("unexpected EOF in list")
        else
            return atom(String(token))
        end
    end
    token1 = next_token(inport)
    return token1 === eof_object ? eof_object : read_ahead(token1)
end

function fix_for_macro(x)
    if isa(x, String)
      if contains(String(x), "define-macro")
        global_env["cons"] = (x, y) -> y==[] ? x : isa(x, Symbol2) ? [x, y...] : [x, y]
        global_env["car"] = x -> [first(x)]
      end
    end
end

function atom(token::String)
    if token == "#t"
        return true
    elseif token == "#f"
        return false
    elseif startswith(token, "\"")
        return unescape_string(token[2:end-1])
    else
        try
            return parse(Int, token)
        catch
            try
                return parse(Float64, token)
            catch
                try
                    return parse(Complex{Float64}, replace(token, "i" => "im"))
                catch
                    return Sym(token, symbol_table=symbol_table)
                end
            end
        end
    end
end

function parse2(inport::Union{String, InPort})

    if isa(inport, String)
        fix_for_macro(inport)
        inport = InPort(IOBuffer(inport))
    end
    return expand(read2(inport), toplevel=true)
end

parse2 (generic function with 1 method)

In [ ]:
tests = [
    "(quote (testing 1 (2.0) -3.14e159)) => (testing 1 (2.0) -3.14e+159)",
    "(+ 2 2) => 4",
    "(+ (* 2 100) (* 1 10)) => 210",
    "(if (> 6 5) (+ 1 1) (+ 2 2)) => 2",
    "(if (< 6 5) (+ 1 1) (+ 2 2)) => 4",
    "(define x 3) => None",
    "x => 3",
    "(+ x x) => 6",
    "(begin (define x 1) (set! x (+ x 1)) (+ x 1)) => 3",
    "((lambda (x) (+ x x)) 5) => 10",
    "(define twice (lambda (x) (* 2 x))) => None",
    "(twice 5) => 10",
    "(define compose (lambda (f g) (lambda (x) (f (g x))))) => None",
    "((compose list twice) 5) => (10)",
    "(define repeat (lambda (f) (compose f f))) => None",
    "((repeat twice) 5) => 20",
    "((repeat (repeat twice)) 5) => 80",
    "(define fact (lambda (n) (if (<= n 1) 1 (* n (fact (- n 1)))))) => None",
    "(fact 3) => 6",
    "(fact 50) => 30414093201713378043612608166064768844377641568960512000000000000",
    "(define abs (lambda (n) ((if (> n 0) + -) 0 n))) => None",
    "(list (abs -3) (abs 0) (abs 3)) => (3 0 3)",
    "(define combine (lambda (f) (lambda (x y) (if (null? x) (quote ()) (f (list (car x) (car y)) ((combine f) (cdr x) (cdr y))))))) => None",
    "(define zip (combine cons)) => None",
    "(zip (list 1 2 3 4) (list 5 6 7 8)) => ((1 5) (2 6) (3 7) (4 8))",
    "(define riff-shuffle (lambda (deck) (begin (define take (lambda (n seq) (if (<= n 0) (quote ()) (cons (car seq) (take (- n 1) (cdr seq)))))) (define drop (lambda (n seq) (if (<= n 0) seq (drop (- n 1) (cdr seq))))) (define mid (lambda (seq) (/ (length seq) 2))) ((combine append) (take (mid deck) deck) (drop (mid deck) deck))))) => None",
    "(riff-shuffle (list 1 2 3 4 5 6 7 8)) => (1 5 2 6 3 7 4 8)",
    "((repeat riff-shuffle) (list 1 2 3 4 5 6 7 8)) => (1 3 5 7 2 4 6 8)",
    "(riff-shuffle (riff-shuffle (riff-shuffle (list 1 2 3 4 5 6 7 8)))) => (1 2 3 4 5 6 7 8)"
]

# Assuming the 'parse' function and 'tests' list are already defined as per previous discussions

println("Parser Tests")
for test in tests
    # Splitting each test case into the input part and the expected output part
    parts = split(test, " => ")
    input_part = String(parts[1])

    println("Parse>>> ", input_part)
    println(parse2(input_part))
    println("")
end

Parser Tests
Parse>>> (quote (testing 1 (2.0) -3.14e159))
Any[Symbol2("quote"), Any[Symbol2("testing"), 1, Any[2.0], -3.14e159]]

Parse>>> (+ 2 2)
Any[Symbol2("+"), 2, 2]

Parse>>> (+ (* 2 100) (* 1 10))
Any[Symbol2("+"), Any[Symbol2("*"), 2, 100], Any[Symbol2("*"), 1, 10]]

Parse>>> (if (> 6 5) (+ 1 1) (+ 2 2))
Any[Symbol2("if"), Any[Symbol2(">"), 6, 5], Any[Symbol2("+"), 1, 1], Any[Symbol2("+"), 2, 2]]

Parse>>> (if (< 6 5) (+ 1 1) (+ 2 2))
Any[Symbol2("if"), Any[Symbol2("<"), 6, 5], Any[Symbol2("+"), 1, 1], Any[Symbol2("+"), 2, 2]]

Parse>>> (define x 3)
Any[Symbol2("define"), Symbol2("x"), 3]

Parse>>> x
Symbol2("x")

Parse>>> (+ x x)
Symbol2[Symbol2("+"), Symbol2("x"), Symbol2("x")]

Parse>>> (begin (define x 1) (set! x (+ x 1)) (+ x 1))
Any[Symbol2("begin"), Any[Symbol2("define"), Symbol2("x"), 1], Any[Symbol2("set!"), Symbol2("x"), Any[Symbol2("+"), Symbol2("x"), 1]], Any[Symbol2("+"), Symbol2("x"), 1]]

Parse>>> ((lambda (x) (+ x x)) 5)
Any[Any[Symbol2("lambda"), Any[Symbol2("x

## Env.jl

In [ ]:
mutable struct Env
    mappings::Dict{Any, Any}
    outer::Union{Env, Nothing}

    function Env(parms=(), args=(), outer=nothing)
        new_env = new(Dict{Any, Any}(), outer)

        if isa(parms, Symbol2)
            new_env.mappings[parms.name] = args
        else
            # Assume parms is a collection of parameters
            if length(args) != length(parms)
                throw(TypeError("Expected $(length(parms_array)), given $(length(args_array))"))
            end
            # Bind each parameter to its corresponding argument
            for (parm, arg) in zip(parms, args)
                new_env.mappings[parm.name] = arg
            end
        end
        return new_env
    end
end

Base.getindex(env::Env, key) = get(env.mappings, key, nothing)
Base.setindex!(env::Env, value, key) = setindex!(env.mappings, value, key)

struct LookupError <: Exception
    msg::String
end

function find(env::Env, var)
    # Check if the variable is in the current environment
    if haskey(env.mappings, var.name)
        return env
    elseif env.outer === nothing
        # If there's no outer environment, throw an error indicating the variable was not found
        throw(KeyError("Variable $var not found"))
    else
        # Recursively search in the outer environment
        return find(env.outer, var)
    end
end


# Define a mutable struct to use for our custom continuation exception
mutable struct ContinuationException <: Exception
    retval::Any  # Store the return value here
end

# This function will be passed to the user-defined procedure to allow escaping
function throw_continuation(retval)
    throw(ContinuationException(retval))  # Throw the exception with the retval
end

# The callcc function, which simulates call-with-current-continuation
function callcc(proc)
    try
        return proc(throw_continuation)  # Pass the throw function to the user procedure
    catch e
        if isa(e, ContinuationException)
            return e.retval  # Return the stored retval when the specific exception is caught
        else
            rethrow(e)  # If any other exception occurs, rethrow it
        end
    end
end

callcc (generic function with 1 method)

In [ ]:
islist(x) = isa(x, Array)
isbool(x) = isa(x, Bool)
issymbol(x) = isa(x, Symbol)
ispair(x) = isa(x, Array) && length(x) == 2
isnumber(x) = isa(x, Number)
isstring(x) = isa(x, AbstractString)
isio(x) = isa(x, IO)
cons(x, y) = [x; y]

function add_globals(env::Env)
    # Math and logical operators
    env["+"] = +
    env["-"] = -
    env["*"] = *
    env["/"] = /

    env[">"] = >
    env["<"] = <
    env[">="] = >=
    env["<="] = <=
    env["="] = ==

    # Type-checking functions
    env["list?"] = islist
    env["boolean?"] = isbool
    env["symbol?"] = issymbol
    env["pair?"] = ispair
    env["number?"] = isnumber
    env["string?"] = isstring
    env["port?"] = isio

    # List operations
    env["not"] = (x) -> !x
    env["cons"] = cons
    env["car"] = x -> first(x)
    env["cdr"] = x -> x[2:end]
    env["append"] = (x, y...) -> [x;y...]
    env["list"] = (x...) -> [x...]
    env["null?"] = isempty
    env["sqrt"] = (x) -> if x>=0 sqrt(x) else sqrt(Complex(x)) end

    # File operations (simplified examples)
    env["open-input-file"] = open
    env["close-input-port"] = close
    env["open-output-file"] = f -> open(f, "w")
    env["close-output-port"] = close

    # I/O operations
    env["read"] = read
    env["write"] = write
    env["display"] = println

    # Other functionalities (apply, eval, load, call/cc) need custom implementation
    env["apply"] = (proc, args) -> proc(args...)
    env["eval"] = (x) -> eval2(expand(x))
    env["load"] = (fn) -> load(fn)
    env["call/cc"] = callcc
    env["length"] = length

    return env
end

global_env = add_globals(Env())

Env(Dict{Any, Any}("read" => read, "null?" => isempty, "close-output-port" => close, ">" => (>), "not" => var"#73#83"(), "sqrt" => var"#78#88"(), "length" => length, "<" => (<), "cdr" => var"#75#85"(), "display" => println…), nothing)

## Eval.jl

In [ ]:
function eval2(x, env=global_env)

    while true
        if isa(x, Symbol2)  # Variable reference or built-in operator
            var_name = x.name
            # Attempt to find the variable in the environment chain
            found_env = find(env, x)
            if found_env !== nothing
                return found_env.mappings[var_name]
            elseif haskey(global_env.mappings, var_name)  # Check the global environment for built-in functions/operators
                return global_env.mappings[var_name]
            else
                throw(KeyError("Variable or operator $var_name not found"))
            end

        elseif !isa(x, Vector)  # Constant literal
            if x==1
              return true
            else
              return x
            end

        elseif x[1] === _quote  # (quote exp)
            return x[2]

        elseif x[1] === _if  # (if test conseq alt)
            test, conseq, alt = x[2], x[3], x[4]
            x = eval2(test, env) ? conseq : alt

        elseif x[1] === _set  # (set! var exp)
            var, exp = x[2], x[3]
            find(env, var)[var.name] = eval2(exp, env)
            return nothing

        elseif x[1] === _define  # (define var exp)
            var, exp = x[2], x[3]
            env.mappings[var.name] = eval2(exp, env)
            return nothing

        elseif x[1] === _lambda  # (lambda (var*) exp)
            vars, exp = x[2], x[3]
            return Procedure(vars, exp, env)

        elseif x[1] === _begin  # (begin exp+)
            for exp in x[2:end-1]
                eval2(exp, env)
            end
            x = x[end]

        else  # (proc exp*)z
            proc = eval2(x[1], env)
            exps = [eval2(exp, env) for exp in x[2:end]]
            if isa(proc, Procedure)
                x = proc.exp
                env = Env(proc.parms, exps, proc.env)
            else
                return proc(exps...)
            end
        end

    end
end

# Represent a Scheme procedure (lambda)
struct Procedure
    parms::Any  # Parameters of the lambda
    exp::Any  # Body of the lambda
    env::Env  # Environment where the lambda was defined
end

# Make Procedure instances callable
function (proc::Procedure)(args...)
    # Create a new environment that extends proc.env with bindings from parms to args
    lambda_env = Env(proc.parms, args, proc.env)
    return eval2(proc.exp, lambda_env)  # Evaluate the procedure's body in the new environment
end

In [ ]:
tests = [
    "(quote (testing 1 (2.0) -3.14e159)) => (testing 1 (2.0) -3.14e+159)",
    "(+ 2 2) => 4",
    "(+ (* 2 100) (* 1 10)) => 210",
    "(if (> 6 5) (+ 1 1) (+ 2 2)) => 2",
    "(if (< 6 5) (+ 1 1) (+ 2 2)) => 4",
    "(define x 3) => nothing",
    "x => 3",
    "(+ x x) => 6",
    "(begin (define x 1) (set! x (+ x 1)) (+ x 1)) => 3",
    "((lambda (x) (+ x x)) 5) => 10",
    "(define twice (lambda (x) (* 2 x))) => nothing",
    "(twice 5) => 10",
    "(define compose (lambda (f g) (lambda (x) (f (g x))))) => nothing",
    "((compose list twice) 5) => (10)",
    "(define repeat (lambda (f) (compose f f))) => nothing",
    "((repeat twice) 5) => 20",
    "((repeat (repeat twice)) 5) => 80",
    "(define fact (lambda (n) (if (<= n 1) 1 (* n (fact (- n 1)))))) => nothing",
    "(fact 3) => 6",
    "(fact 10) => 3628800",
    "(define abs (lambda (n) ((if (> n 0) + -) 0 n))) => nothing",
    "(list (abs -3) (abs 0) (abs 3)) => (3 0 3)",
    "(define combine (lambda (f) (lambda (x y) (if (null? x) (quote ()) (f (list (car x) (car y)) ((combine f) (cdr x) (cdr y))))))) => nothing",
    "(define zip (combine cons)) => nothing",
    "(zip (list 1 2 3 4) (list 5 6 7 8)) => ((1 5) (2 6) (3 7) (4 8))",
    "(define riff-shuffle (lambda (deck) (begin (define take (lambda (n seq) (if (<= n 0) (quote ()) (cons (car seq) (take (- n 1) (cdr seq)))))) (define drop (lambda (n seq) (if (<= n 0) seq (drop (- n 1) (cdr seq))))) (define mid (lambda (seq) (/ (length seq) 2))) ((combine append) (take (mid deck) deck) (drop (mid deck) deck))))) => None",
    "(riff-shuffle (list 1 2 3 4 5 6 7 8)) => (1 5 2 6 3 7 4 8)",
    "((repeat riff-shuffle) (list 1 2 3 4 5 6 7 8)) => (1 3 5 7 2 4 6 8)",
    "(riff-shuffle (riff-shuffle (riff-shuffle (list 1 2 3 4 5 6 7 8)))) => (1 2 3 4 5 6 7 8)",
    "(define (twice x) (* 2 x)) => nothing",
    "(twice 2) => 4",
    "(define lyst (lambda items items)) => nothing",
    "(lyst 1 2 3 (+ 2 2)) => (1 2 3 4)",
    "(if 1 2) => 2",
    "(if (= 3 4) 2) => nothing",
    "(define ((account bal) amt) (set! bal (+ bal amt)) bal) => nothing",
    "(define a1 (account 100)) => nothing",
    "(a1 0) => 100",
    "(a1 10) => 110",
    "(a1 10) => 120",
    "(define (newton guess function derivative epsilon) (define guess2 (- guess (/ (function guess) (derivative guess)))) (if (< (abs (- guess guess2)) epsilon) guess2 (newton guess2 function derivative epsilon))) => None",
    "(define (square-root a) (newton 1 (lambda (x) (- (* x x) a)) (lambda (x) (* 2 x)) 1e-8)) => nothing",
    "(> (square-root 200.) 14.14213) => #t",
    "(< (square-root 200.) 14.14215) => #t",
    "(= (square-root 200.) (sqrt 200.)) => #t",
    "(define (sum-squares-range start end) (define (sumsq-acc start end acc) (if (> start end) acc (sumsq-acc (+ start 1) end (+ (* start start) acc)))) (sumsq-acc start end 0)) => None",
    "(sum-squares-range 1 3000) => 9004500500",
    "(call/cc (lambda (throw) (+ 5 (* 10 (throw 1))))) ;; throw => 1",
    "(call/cc (lambda (throw) (+ 5 (* 10 1)))) ;; do not throw => 15",
    "(call/cc (lambda (throw) (+ 5 (* 10 (call/cc (lambda (escape) (* 100 (escape 3)))))))) ; 1 level => 35",
    "(call/cc (lambda (throw) (+ 5 (* 10 (call/cc (lambda (escape) (* 100 (throw 3)))))))) ; 2 levels => 3",
    "(call/cc (lambda (throw) (+ 5 (* 10 (call/cc (lambda (escape) (* 100 1))))))) ; 0 levels => 1005",
    "(* 1i 1i) => (-1+0i)",
    "(sqrt -1) => 1i",
    "(let ((a 1) (b 2)) (+ a b)) => 3",
#    "(and 1 2 3) => 3",
#    "(and (> 2 1) 2 3) => 3",
#    "(and) => #t",
#    "(and (> 2 1) (> 2 3)) => #f",
    "(define-macro unless (lambda args `(if (not ,(car args)) (begin ,@(cdr args))))) ; test ` => nothing",
    "(unless (= 2 (+ 1 1)) (display 2) 3 4) => nothing",
    "(unless (= 4 (+ 1 1)) 4) => 4",
    "(quote x) => x",
    "(quote (1 2 three)) => (1 2 three)"
]

println("scm simulation: Tests")
for test in tests
    # Splitting each test case into the input part and the expected output part
    parts = split(test, " => ")
    input_part = String(parts[1])
    output_part = String(parts[2])

    println("jscm>>> ", input_part, "   ; scheme expected output: ", output_part)
    println(eval2(parse2(input_part)))
    println("")
end

scm simulation: Tests
jscm>>> (quote (testing 1 (2.0) -3.14e159))   ; scheme output: (testing 1 (2.0) -3.14e+159)
Any[Symbol2("testing"), 1, Any[2.0], -3.14e159]

jscm>>> (+ 2 2)   ; scheme output: 4
4

jscm>>> (+ (* 2 100) (* 1 10))   ; scheme output: 210
210

jscm>>> (if (> 6 5) (+ 1 1) (+ 2 2))   ; scheme output: 2
2

jscm>>> (if (< 6 5) (+ 1 1) (+ 2 2))   ; scheme output: 4
4

jscm>>> (define x 3)   ; scheme output: nothing
nothing

jscm>>> x   ; scheme output: 3
3

jscm>>> (+ x x)   ; scheme output: 6
6

jscm>>> (begin (define x 1) (set! x (+ x 1)) (+ x 1))   ; scheme output: 3
3

jscm>>> ((lambda (x) (+ x x)) 5)   ; scheme output: 10
10

jscm>>> (define twice (lambda (x) (* 2 x)))   ; scheme output: nothing
nothing

jscm>>> (twice 5)   ; scheme output: 10
10

jscm>>> (define compose (lambda (f g) (lambda (x) (f (g x)))))   ; scheme output: nothing
nothing

jscm>>> ((compose list twice) 5)   ; scheme output: (10)
[10]

jscm>>> (define repeat (lambda (f) (compose f f)))   ; scheme 

In [ ]:
errorTests = [
    "() => SyntaxError (): wrong length",
    "(set! x) => SyntaxError (set! x): wrong length",
    "(define 3 4) => SyntaxError (define 3 4): can define only a symbol",
    "(quote 1 2) => SyntaxError (quote 1 2): wrong length",
    "(if 1 2 3 4) => SyntaxError (if 1 2 3 4): wrong length",
    "(lambda 3 3) => SyntaxError (lambda 3 3): illegal lambda argument list",
    "(lambda (x)) => SyntaxError (lambda (x)): wrong length",
    "(if (= 1 2) (define-macro a 'a) (define-macro a 'b)) => SyntaxError (define-macro a (quote a)): define-macro only allowed at top level",
    "(twice 2 2) => TypeError expected (x), given (2 2)",
    "(let ((a 1) (b 2 3)) (+ a b)) => SyntaxError (let ((a 1) (b 2 3)) (+ a b)): illegal binding list"
]

10-element Vector{String}:
 "() => SyntaxError (): wrong length"
 "(set! x) => SyntaxError (set! x): wrong length"
 "(define 3 4) => SyntaxError (define 3 4): can define only a symbol"
 "(quote 1 2) => SyntaxError (quote 1 2): wrong length"
 "(if 1 2 3 4) => SyntaxError (if 1 2 3 4): wrong length"
 "(lambda 3 3) => SyntaxError (lambda 3 3): illegal lambda argument list"
 "(lambda (x)) => SyntaxError (lambda (x)): wrong length"
 "(if (= 1 2) (define-macro a 'a) (define-m" ⋯ 52 bytes ⋯ "): define-macro only allowed at top level"
 "(twice 2 2) => TypeError expected (x), given (2 2)"
 "(let ((a 1) (b 2 3)) (+ a b)) => SyntaxError (let ((a 1) (b 2 3)) (+ a b)): illegal binding list"

## Repl.jl

In [ ]:
# Define the load function that evaluates every expression from a file
function load(filename)
    inport = InPort(open(filename, "r"))
    try
        repl(nothing, inport, stdout)
    finally
        close(inport.file)  # Ensure the file is closed after reading
    end
end

# Define the REPL function
function repl(prompt::Union{Nothing, String}="jscm>>> ", inport::InPort=InPort(stdin), out::IO=stdout)
    println(stderr, "Basic Julia Scheme Interpreter")
    while true
        try
            if !isnothing(prompt)
                write(stderr, prompt)
            end
            x = parse2(inport)
            if x === eof_object
                return
            end
            val = eval2(x)
            if !isnothing(val)
                println(out, to_string(val))
            end
        catch e
            println(stderr, "$(typeof(e).__name__): $e")
        end
    end
end


repl (generic function with 4 methods)

In [ ]:
repl()

Basic Julia Scheme Interpreter
jscm>>> 